# 11 Jan 2022

In [20]:
import argparse
import os
import re
import sys
import time
import requests
from urllib.parse import urlparse
import bs4
from bs4 import BeautifulSoup
from bs4.element import NavigableString
#bs4.element.NavigableString
import pandas as pd 
#from convert import shortToBytes, bytesToShort
import joblib

In [2]:
#https://www.ampreviews.net/index.php?forums/discussion-new-york-city-manhattan.70/
example_url = 'https://www.ampreviews.net/index.php?threads/people-with-rubmaps-accounts.9374/'
example_url = 'https://www.google.com'
response = requests.get(example_url, cookies=None, headers=None, timeout=10)

In [3]:
soup = BeautifulSoup(response.text, 'html.parser')

# Scraper Code 

In [4]:
QUOTE_CSS_CLASS = 'div.bbCodeBlock--quote'

def extract_ampreviews_data(url):
    response = requests.get(url, cookies=None, headers=None, timeout=10)
    soup = BeautifulSoup(response.text, 'html.parser')
    msgs = soup.select('article.message--post')
    print('extracting reviews for url: ', url, f'found {len(msgs)} messages')
    data = []
    for msg in msgs:
        username = msg['data-author']
        user_id = msg.select_one('a.username')['data-user-id']
        post_id = msg['data-content'] # this is the msg id within the thread
        msg_time = msg.select_one('div.message-date time')['title']
        msg_epoch_time = msg.select_one('div.message-date time')['data-time']
        #msg_time  = msg.select('time')[1] # the msg timestamp is 2nd time (first is when author joined forum)

        msg_body = msg.select_one('article.message-body')

        if msg_body.select_one(QUOTE_CSS_CLASS):
            quote_src_postid = msg_body.select_one('a.bbCodeBlock-sourceJump')['href']
            quoted_text = msg_body.select_one('div.bbCodeBlock-expandContent').text
            # actual message contained in top level div
            msg_text = msg_body.find('div').contents
            msg_text= ''.join([x for x in msg_body.find('div').contents if isinstance(x, bs4.element.NavigableString)])
        else:
            quote_src_postid = None
            quoted_text = None
            msg_text = msg_body.text
        data.append( [username, user_id, post_id, msg_time, msg_text, quote_src_postid, quoted_text] )
    df = pd.DataFrame()
    if len(msgs) > 0:
        df = pd.DataFrame(data)
        df.columns = ['username', 'user_id', 'post_id', 'msg_time', 'msg_text', 'quote_src_postid', 'quoted_text']
        print('done extracting reviews for url: ', url, f' returning dataframe of shape {df.shape}')
        #print(df.username)
    return df

In [5]:
data = extract_ampreviews_data(example_url)

extracting reviews for url:  https://www.google.com found 0 messages


In [6]:
#https://www.ampreviews.net/index.php?forums/discussion-new-york-city-manhattan.70,
#https://www.ampreviews.net/index.php?forums/boardwide-general-topics.61,

In [9]:
url_list = pd.read_csv('./list_urls.txt', comment='#')
url_list.columns=['url','']
for url in url_list.to_numpy():
    print(url)

['https://www.ampreviews.net/index.php?threads/lks-lsc-bgc-pick-one-from-each-org.116680/'
 nan]
['https://www.ampreviews.net/index.php?threads/review-mj-spa.116611/' nan]
['https://www.ampreviews.net/index.php?threads/new-skylink-gals.116634/'
 nan]
['https://www.ampreviews.net/index.php?threads/review-365-spa-lisa.116640/'
 nan]
['https://www.ampreviews.net/index.php?threads/review-lkp-suzie-jenny.116453/'
 nan]
['https://www.ampreviews.net/index.php?threads/review-hibiki-pleasure-palace.115472/'
 nan]
['https://www.ampreviews.net/index.php?threads/what-it%E2%80%99s-up-with-punta-cana.116679/'
 nan]
['https://www.ampreviews.net/index.php?threads/review-roxy-just-don-nyaf.116247'
 nan]
['https://www.ampreviews.net/index.php?threads/review-china-accupressure-tiffany.116639'
 nan]
['https://www.ampreviews.net/index.php?threads/review-roxy-at-nyaf.116678'
 nan]
['https://www.ampreviews.net/index.php?threads/review-spinner-rita.116675'
 nan]
['https://www.ampreviews.net/index.php?threads/

In [10]:
!ls

0 - EDA.ipynb
ampreviews_scraped_forum_data.csv
ampreviews_scraped_forum_dataframe.joblib
bs4_example_pt2.ipynb
list_urls.txt
notes_on_ampreviews_structure.md
notes_on_ampreviews_structure.txt
topic-modeling-lda.ipynb


In [11]:
time.sleep(0.01) # sleep 10 msec
print(time.ctime())

Wed Jan 11 11:57:08 2023


In [12]:
list_data = []
print(time.ctime())
for i, url in enumerate(url_list['url']):
    print(i, '/',  len(url_list), ',',  url)
    #if i > 4:
       #break
    data =  extract_ampreviews_data(url)
    if len(data) != 0:
        list_data.append(data)
    time.sleep(0.1) 
print(time.ctime())

Wed Jan 11 11:57:09 2023
0 / 39 , https://www.ampreviews.net/index.php?threads/lks-lsc-bgc-pick-one-from-each-org.116680/
extracting reviews for url:  https://www.ampreviews.net/index.php?threads/lks-lsc-bgc-pick-one-from-each-org.116680/ found 11 messages
done extracting reviews for url:  https://www.ampreviews.net/index.php?threads/lks-lsc-bgc-pick-one-from-each-org.116680/  returning dataframe of shape (11, 7)
1 / 39 , https://www.ampreviews.net/index.php?threads/review-mj-spa.116611/
extracting reviews for url:  https://www.ampreviews.net/index.php?threads/review-mj-spa.116611/ found 5 messages
done extracting reviews for url:  https://www.ampreviews.net/index.php?threads/review-mj-spa.116611/  returning dataframe of shape (5, 7)
2 / 39 , https://www.ampreviews.net/index.php?threads/new-skylink-gals.116634/
extracting reviews for url:  https://www.ampreviews.net/index.php?threads/new-skylink-gals.116634/ found 14 messages
done extracting reviews for url:  https://www.ampreviews.net

KeyboardInterrupt: 

In [30]:
list_data[0]

,username,user_id,post_id,msg_time,msg_text,quote_src_postid,quoted_text
0,RX78-2,139612,post-845741,"Jul 11, 2022 at 11:04 AM","\nIs hard to narrow down to ""the one"" , I do f...",None,None
1,Juanjr650,167918,post-845868,"Jul 11, 2022 at 2:11 PM","\nI will take Joah, Binna and Irene over any g...",None,None
2,Paradise 408,175403,post-845969,"Jul 11, 2022 at 4:15 PM","\nMila, Adela, Daya at this time\n \n",None,None
3,atanto,84110,post-846002,"Jul 11, 2022 at 5:16 PM","\nSeri, soda, Irene\n \n",None,None
4,tomasanderson161,130665,post-846006,"Jul 11, 2022 at 5:28 PM",\nLKS - Adel\nLSC - Rose\nBGC - Margo\n \n,None,None
5,Rapscallion,98767,post-846091,"Jul 11, 2022 at 7:54 PM",\nDamn some dudes are lucky enough to be verif...,None,None
6,Logan2k,139217,post-846100,"Jul 11, 2022 at 8:15 PM",\nProbably old schoolers that got in before th...,None,None
7,anthonyjacobssf,112692,post-846166,"Jul 11, 2022 at 10:13 PM",\nLKS - Jennie (that foursome with Sarah and N...,None,None
8,hotdragon731,174740,post-846526,"Jul 12, 2022 at 1:38 PM",I'm new here. May I get link for all 3 orgs a...,/index.php?goto/post&id=845741,"\n\t\t\tIs hard to narrow down to ""the one"" , ..."
9,Juanjr650,167918,post-846605,"Jul 12, 2022 at 2:52 PM",\nGo here for links\nBayareakoreangirls.com\n \n,None,None


In [27]:
list_data

[            username user_id      post_id                  msg_time  \
 0             RX78-2  139612  post-845741  Jul 11, 2022 at 11:04 AM   
 1          Juanjr650  167918  post-845868   Jul 11, 2022 at 2:11 PM   
 2       Paradise 408  175403  post-845969   Jul 11, 2022 at 4:15 PM   
 3             atanto   84110  post-846002   Jul 11, 2022 at 5:16 PM   
 4   tomasanderson161  130665  post-846006   Jul 11, 2022 at 5:28 PM   
 5        Rapscallion   98767  post-846091   Jul 11, 2022 at 7:54 PM   
 6            Logan2k  139217  post-846100   Jul 11, 2022 at 8:15 PM   
 7    anthonyjacobssf  112692  post-846166  Jul 11, 2022 at 10:13 PM   
 8       hotdragon731  174740  post-846526   Jul 12, 2022 at 1:38 PM   
 9          Juanjr650  167918  post-846605   Jul 12, 2022 at 2:52 PM   
 10   herrwaldorivera   53003  post-848304  Jul 15, 2022 at 12:56 AM   
 
                                              msg_text  \
 0   \nIs hard to narrow down to "the one" , I do f...   
 1   \nI will take

In [31]:
forum_data = pd.concat(list_data)

In [33]:
forum_data.describe()
forum_data.sample()

,username,user_id,post_id,msg_time,msg_text,quote_src_postid,quoted_text
7,Chief5665,5953,post-845733,"Jul 11, 2022 at 10:47 AM",\nDid Hibiki move over to JPK? or back to Japa...,None,None


In [16]:
import joblib
joblib.dump(forum_data, './ampreviews_scraped_forum_dataframe.joblib')

['./ampreviews_scraped_forum_dataframe.joblib']

In [44]:
forum_data.msg_text.to_csv('./ampreviews_text.csv', index=False, encoding='utf-8')

In [45]:
# confirm saved correctly
tmp = pd.read_csv('./ampreviews_text.csv')
tmp

,msg_text
0,"\nIs hard to narrow down to ""the one"" , I do f..."
1,"\nI will take Joah, Binna and Irene over any g..."
2,"\nMila, Adela, Daya at this time\n \n"
3,"\nSeri, soda, Irene\n \n"
4,\nLKS - Adel\nLSC - Rose\nBGC - Margo\n \n
5,\nDamn some dudes are lucky enough to be verif...
6,\nProbably old schoolers that got in before th...
7,\nLKS - Jennie (that foursome with Sarah and N...
8,I'm new here. May I get link for all 3 orgs a...
9,\nGo here for links\nBayareakoreangirls.com\n \n


In [19]:
# Check saved correctly (e.g. in case of git pull issues)
#df2 = joblib.load('./ampreviews_scraped_forum_dataframe.joblib')
#df2

,username,user_id,post_id,msg_time,msg_text,quote_src_postid,quoted_text
0,RX78-2,139612,post-845741,"Jul 11, 2022 at 11:04 AM","\nIs hard to narrow down to ""the one"" , I do f...",None,None
1,Juanjr650,167918,post-845868,"Jul 11, 2022 at 2:11 PM","\nI will take Joah, Binna and Irene over any g...",None,None
2,Paradise 408,175403,post-845969,"Jul 11, 2022 at 4:15 PM","\nMila, Adela, Daya at this time\n \n",None,None
3,atanto,84110,post-846002,"Jul 11, 2022 at 5:16 PM","\nSeri, soda, Irene\n \n",None,None
4,tomasanderson161,130665,post-846006,"Jul 11, 2022 at 5:28 PM",\nLKS - Adel\nLSC - Rose\nBGC - Margo\n \n,None,None
5,Rapscallion,98767,post-846091,"Jul 11, 2022 at 7:54 PM",\nDamn some dudes are lucky enough to be verif...,None,None
6,Logan2k,139217,post-846100,"Jul 11, 2022 at 8:15 PM",\nProbably old schoolers that got in before th...,None,None
7,anthonyjacobssf,112692,post-846166,"Jul 11, 2022 at 10:13 PM",\nLKS - Jennie (that foursome with Sarah and N...,None,None
8,hotdragon731,174740,post-846526,"Jul 12, 2022 at 1:38 PM",I'm new here. May I get link for all 3 orgs a...,/index.php?goto/post&id=845741,"\n\t\t\tIs hard to narrow down to ""the one"" , ..."
9,Juanjr650,167918,post-846605,"Jul 12, 2022 at 2:52 PM",\nGo here for links\nBayareakoreangirls.com\n \n,None,None


In [ ]:
# in order to automate it:
